In [ ]:
#import required packages
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas
import panel as pn
import plotly.express as px
import plot_functions as pf

from bokeh.models.formatters import NumeralTickFormatter
formatter = NumeralTickFormatter(format="0,0")

In [ ]:
# Read processed data
scot_mfg_df = pd.read_pickle('interim/scot_adv_mfg_df.pkl')
scot_mfg_df.rename(columns = {"TotalDealroomFundingMillionEuros":"DealroomFunding"}, inplace = True)
scot_mfg_df['DealroomFunding'] = scot_mfg_df['DealroomFunding'].replace(np.nan, 0)


In [ ]:

df_rtics = pd.DataFrame({})
cols =['Robotics and Automation','Forging','Forming','Computer Aided Manufacturing','Prototyping',
       'Photonics','Digital Twins','Digital Design','Mixed Reality','Renewables']
for col in cols:

    data = scot_mfg_df["RTICsubsectors"].str.contains(col)
    #data=scot_mfg_df["RTICsubsectors"].isin(cols)
    #data = scot_mfg_df['RTICsubsectors'].str.split(',').explode().isin([col]).groupby(level=0).any()

    df = scot_mfg_df[data]

    df = scot_mfg_df[data].where(scot_mfg_df["BestEstimateEmployees"]>0)
    df = df.dropna(subset=['BestEstimateEmployees'])
    df.reset_index(drop=True, inplace=True)

    s = pd.Series(np.tile([col], len(df)))
    df['RTIC'] = s

    df['yearly growth rate'] = ((df.Turnover_2017-df.Turnover_2016)+(df.Turnover_2018-df.Turnover_2017)+(df.Turnover_2019-df.Turnover_2018)+(df.Turnover_2020-df.Turnover_2019))/df.BestEstimateTurnover    
    df_rtics = pd.concat([df_rtics, df])

df_rtics.reset_index(drop=True, inplace=True)
df_rtics

In [ ]:
process_df2 = df_rtics.mask(df_rtics["BestEstimateGrowthPercentagePerYear"]==0)
process_df2['Incorporation Year']= pd.to_datetime(df_rtics.IncorporationDate).dt.year
process_df2.rename(columns = {"BestEstimateGrowthPercentagePerYear":"Growth Per Year"}, inplace = True)
process_df2 = process_df2.dropna(subset=['Growth Per Year'])
process_df2.reset_index(drop=True, inplace=True)

def plot_scatter():
    
    fig_a = process_df2.hvplot(x='Incorporation Year', y='Growth Per Year', by='RTIC', kind='scatter', alpha=0.6, persist=True
                       ,ylim=[-1,1],width = 1600,height = 600, hover_cols=['Companyname', 'RTICsubsectors'])
    return fig_a

In [ ]:
def sub_plot_scatter():
    fig_b = process_df2.hvplot(x='Incorporation Year', y='Growth Per Year', by='RTIC', kind='scatter',
                   ylim=[-1,1],subplots=True,width = 400,height = 300, hover_cols=['Companyname', 'RTICsubsectors'])
    return fig_b

In [ ]:
# Plot total count for each sector and cumulative percentage
db_explanation_pane = pn.pane.Markdown("""
# Incorporation Year vs. growth per year
""", width=500)
#count_db_size_chart_interact = pn.interact(counts_sector)
#plot_together = pn.Row(plot_scatter,sub_plot_scatter )
count_db_size_layout = pn.Column(db_explanation_pane, plot_scatter)
db_size_layout = pn.Column(count_db_size_layout, sub_plot_scatter)
db_size_layout.servable()

In [ ]:
def plot_bar_chart():    
    c3_df = pd.DataFrame({})
    c3_df['growth rate (mean)'] = process_df2.groupby('RTIC').agg({"Growth Per Year": "mean"})
    c3_df['growth rate (std)'] = process_df2.groupby('RTIC').agg({"Growth Per Year": np.std})
    c3_df = c3_df.sort_values(by='growth rate (mean)', ascending = True)
    fig_c = c3_df.hvplot.bar(height=600, width=1600, rot=75)
    
    return fig_c

In [ ]:
# Plot total count for each sector and cumulative percentage
db_explanation_pane = pn.pane.Markdown("""
# Growth rate
""", width=200)
#count_db_size_chart_interact = pn.interact(counts_sector)
#plot_together = pn.Row(plot_scatter,sub_plot_scatter )
layout = pn.Column(db_explanation_pane, plot_bar_chart)
layout.servable()

In [ ]:
# Used defined colormap
s='''
    red, yellow, darkmagenta,orange,
    magenta,blue,
    greenyellow, cyan, indianred, indigo,
    darkgoldenrod, darkgreen, orange, darkolivegreen, darkorange,
    darkorchid, darkred, darksalmon, hotpink
    '''
colours = s.split(',')
colours = [c.replace('\n','') for c in colours]
colours = [c.replace(' ','') for c in colours]
colour_map = {rtics:c for rtics,c in zip(set(df_rtics.RTIC), colours)}
colour_map

In [ ]:
pn.extension('plotly')
#Plot estimate turnover of all the recorded companies w.r.t location
rev_geog_pane = pn.pane.Markdown("""
# Registered Location
""", width=1000)

rev_geog_fig = px.scatter_mapbox(df_rtics, lat="latitude", lon="longitude", color=df_rtics.RTIC, color_discrete_map=colour_map, 
                                     mapbox_style='open-street-map',hover_name='Companyname', hover_data=['BestEstimateTurnover'], 
                                     zoom = 8, height=1200, width=1200)
                                    
rev_geog_fig_pane = pn.pane.Plotly(rev_geog_fig)
rev_geog_layout = pn.Column(rev_geog_pane, rev_geog_fig_pane)
rev_geog_layout.servable()